# Store Sales - Time Series Forecasting

Use machine learning to predict grocery sales. [source](https://www.kaggle.com/competitions/store-sales-time-series-forecasting/overview/description)

## Objective

In this Kaggle competition, the goal is to 

> build a model that more accurately predicts the unit sales for thousands of items sold at different Favorita stores.

The evaluation metric for this competition is ***Root Mean Squared Logarithmic Error***.

The `RMSLE` is calculated as:

$$\sqrt{ \frac{1}{n} \sum_{i=1}^n \left(\log (1 + \hat{y}_i) - \log (1 + y_i)\right)^2}$$

where:

- $ n $ is the total number of instances,
     
- $\hat{y}$ is the predicted value of the target for instance (i),
   
- $y_i$ is the actual value of the target for instance (i), and,
 
- $log$ is the natural logarithm.

For each id in the test set, you must predict a value for the sales variable. The file should contain a header and have the following format:

    ```
    id,sales
    3000888,0.0
    3000889,0.0
    3000890,0.0
    3000891,0.0
    3000892,0.0
    etc.
    ```


## Libraries for this research notebook

Extra libraries for this version of notebook:
- ydata-profiling
- tqdm
- connectorx
- pycaret

via below conda or pip install (or copy+paste into Terminal)

In [ ]:
# !conda install ydata-profiling tqdm connectorx pycaret

In [ ]:
# !pip install ydata-profiling tqdm connectorx pycaret

In [ ]:

import pandas as pd
from tqdm.auto import tqdm

# to overcome path issue for src
%reload_ext autoreload
%autoreload 2

from pathlib import Path
import sys

# set the path to the current file
current_file_path = Path().resolve()
print(f"current_file_path is {current_file_path}")

# set the path to the src folder
src_folder_path = current_file_path.parent / 'src'
print(f"src_folder_path is {src_folder_path}")

# add the src folder to the system path
sys.path.append(str(src_folder_path))

from data_loader import DBDataLoader
from logger import logging

## Data Ingestion

Query data from MySQL

In [ ]:
query='SELECT * FROM quito_frm_view'

In [ ]:
from dotenv import dotenv_values
config = dotenv_values()

user = config.get('USERNAME')
password = config.get('PASSWORD')
host = config.get('ENDPOINT')
db = config.get('DBNAME')

In [ ]:
import connectorx as cx

conn=f'mysql://{user}:{password}@{host}/{db}' 
df=cx.read_sql(conn, query) # , partition_on="id", partition_num=100

In [ ]:
from sqlalchemy import create_engine
import MySQLdb

engine = create_engine(f'mysql+mysqldb://{user}:{password}@{host}/{db}')
df_pandas = pd.read_sql(sql=query, con=engine
                        # , chunksize=10000
                        )

# print(f'chunks size: {sys.getsizeof(chunks)}')
# logging.info(f"chunks loaded {sys.getsizeof(chunks)}")

# df_pandas = pd.DataFrame()
# for i in tqdm(range(sys.getsizeof(chunks)), desc='Reading from View'):
#     for chunk in chunks:
#         view_df = pd.concat([df_pandas, chunk])

Notes: for comparisons  planetscale limitations of 20sec and 100000 rows

- MySql Workbench fetch `query` in less than **15sec** for 648648 rows
- connectorx fetch `query` in less than 
  - **18sec** for 648648 rows, with no partitioning
  - **40sec** for 648648 rows, using partitions=10
  - **1min 50sec** for 648648 rows, using partitions=100
- sqlalchemy+pandas fetch `query` in less than **20sec** for 648648 rows, without using chunks

Need to verify IDs with original train.csv. Did our database schema auto-increment everytime we run a fetch with "Select" or upon "create view"? See `id` columns in `df` versus `df_pandas` is different.

df 2nd run: id range from 233706 - 1323002 (3rd run with 0 partitions has same IDs)
df_pandas 1st run: id range from 0 - 1945943

In [ ]:
df_pandas

In [ ]:
df

In [ ]:
df.shape

In [ ]:
df.info()

DF loaded confirm: 1972674 rows × 14 columns

In [ ]:
stmt='select * from VwDump1'

chunks = db.load(query=stmt)

print(f'VwDump1 chunks size: {sys.getsizeof(chunks)}')
logging.info(f"VwDump1 chunks loaded {sys.getsizeof(chunks)}")

view_df = pd.DataFrame()
for i in tqdm(range(sys.getsizeof(chunks)), desc='Reading from View'):
    for chunk in chunks:
        view_df = pd.concat([view_df, chunk])

In [ ]:
view_df.head()

DF loaded confirm: 3000888 rows × 14 columns

In [ ]:
view_df.info()

In [ ]:
# trying out data ingestion with connectorx library
from sqlalchemy import create_engine, text
from dotenv import dotenv_values
config = dotenv_values()

In [ ]:
# create engine to talk to database
engine = create_engine(
    f'mysql+pymysql://'             # dialect + driver
    f'{config.get("USERNAME")}'     # username
    f':{config.get("PASSWORD")}'    # password
    f'@{config.get("ENDPOINT")}'    # host
    f':{config.get("PORT")}'        # port
    f'/{config.get("DBNAME")}'      # database
)

In [ ]:
# establish connection and make the query
with engine.connect() as cnxn:
    with open('../src/scripts/query_data.sql') as f:
        query = text(f.read())
        results = pd.read_sql(query, cnxn)

# runtime 1 min 1.4 secs

In [ ]:
results.shape

In [ ]:
from dotenv import load_dotenv
import os
import mysql.connector

load_dotenv()

In [ ]:
DB_HOST = os.getenv("PS_HOST")
DB_USERNAME = os.getenv("PS_USERNAME")
DB_PASSWORD = os.getenv("PS_PASSWORD")
DB_DATABASE = os.getenv("PS_DATABASE")

In [ ]:
connection = mysql.connector.connect(
    host=DB_HOST,
    user=DB_USERNAME,
    password=DB_PASSWORD,
    database=DB_DATABASE,
    # ssl_verify_identity=True,
    # ssl_ca="/etc/ssl/certs/ca-certificates.crt"
)

In [ ]:
cursor = connection.cursor()

batch_size = 100000
start_id = 0
rows = []

while True:
    
    cursor.execute(
        f"""
        SELECT *
        FROM train
        WHERE id >= {start_id}
        ORDER BY id
        LIMIT {batch_size}
        """
    )
    
    batch = cursor.fetchall()
    if not batch:
        break
    
    rows.extend(batch)
    start_id = batch[-1][0] + 1
    
    if len(rows) >= 3000000:
        break

df = pd.DataFrame.from_records(rows, columns=[desc[0] for desc in cursor.description])

In [ ]:
connection = mysql.connector.connect(
    host=DB_HOST,
    user=DB_USERNAME,
    password=DB_PASSWORD,
    database=DB_DATABASE,
    # ssl_verify_identity=True,
    # ssl_ca="/etc/ssl/certs/ca-certificates.crt"
    connect_timeout=1000
)

cursor = connection.cursor()

cursor.execute(
    f"""
    SET GLOBAL connect_timeout=60;
    """
)

cursor.execute(
    f"""
    SET WORKLOAD = 'olap';
    """
)

cursor.execute(
    f"""
    SELECT * FROM full_df
    """
)

rows = cursor.fetchall()

print(len(rows))

df = pd.DataFrame.from_records(rows, columns=[desc[0] for desc in cursor.description])

cursor.close()
connection.close()

In [ ]:
df.shape

## Data cleaning

In [ ]:
df.dtypes

In [ ]:
# df['date'] = pd.to_datetime(df['date'])

In [ ]:
df.set_index('date', drop=True, inplace=True)

In [ ]:
df['weekday'] = df.index.day_name()

In [ ]:
df.isnull().sum()

In [ ]:
groupby_store = df.groupby(by=['store_nbr', 'family'], group_keys=True).agg('sum', 'mean')

In [ ]:
groupby_store.info()

## Data profile

In [ ]:
# from ydata_profiling import ProfileReport

In [ ]:
# profile = ProfileReport(df, tsmode=True, title="Time-Series EDA Quito City")
# profile.to_notebook_iframe()
# profile.to_file("../artifacts/reports/quito_ProfileReport.html") # AttributeError: 'float' object has no attribute 'shape'

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# Use seaborn style defaults and set the default figure size
sns.set(rc={'figure.figsize':(11, 4)})

In [ ]:
df['sales_sum'].plot();

In [ ]:
df.columns

In [ ]:
cols_plot = ['onpromotion_sum', 'transactions_sum', 'sales_sum']
axes = df[cols_plot].plot(marker='.', alpha=0.5, linestyle='None', figsize=(11, 9), subplots=True)
for ax in axes:
    ax.set_ylabel('Daily Totals (GWh)')

In [ ]:
df_2013 = df.loc['2013']

In [ ]:
df_2013

In [ ]:
ax = df_2013.loc['2013', 'sales_sum'].plot()
ax.set_ylabel('Daily Sales for 2013');

### Seasonality

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(11, 10), sharex=True)
for name, ax in zip(cols_plot, axes):
    sns.boxplot(data=df, x='weekday', y=name, ax=ax)
    ax.set_ylabel('Sum')
    ax.set_title(name)
    # Remove the automatic x-axis label from all but the bottom subplot
    if ax != axes[-1]:
        ax.set_xlabel('')

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(11, 10), sharex=True)
for name, ax in zip(cols_plot, axes):
    sns.boxplot(data=df, x='month', y=name, ax=ax)
    ax.set_ylabel('Sum')
    ax.set_title(name)
    # Remove the automatic x-axis label from all but the bottom subplot
    if ax != axes[-1]:
        ax.set_xlabel('')

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(11, 10), sharex=True)
for name, ax in zip(cols_plot, axes):
    sns.boxplot(data=df, x='store_nbr', y=name, ax=ax)
    ax.set_ylabel('Sum')
    ax.set_title(name)
    # Remove the automatic x-axis label from all but the bottom subplot
    if ax != axes[-1]:
        ax.set_xlabel('')

## Data preprocessing

In [ ]:
df.store_nbr.value_counts()

Notes:

- have outliers needing treatment; need to treat outliers first before we can analyse seasonality for `weekday` and `month`
- selected Store 44 as it has most spread for `transaction_sum` meaning more activity; note assumption that 0 transaction means store is closed as there's no sale on that day
- which tracks with 0 transactions occuring on days with holidays (not included to keep dataset small)

In [ ]:
df_str44 = df[(df.store_nbr == 44)]

In [ ]:
grp_df_str44 = df_str44.groupby(by=['date'], group_keys=True).agg('sum')

In [ ]:
grp_df_str44.drop(columns=['id','store_nbr','year','month','day_of_month'], inplace=True)

In [ ]:
grp_df_str44 = grp_df_str44.asfreq('D')

## autoML with pycaret

EDA and ML


In [ ]:
grp_df_str44.info()

In [ ]:
# check installed version
import pycaret
pycaret.__version__

In [ ]:
# import pycaret time series and init setup
from pycaret.time_series import *
s = setup(grp_df_str44,  
            target='sales_sum', 
            fh = 28, 
            session_id = 123, 
            profile=True,
            numeric_imputation_exogenous='mean',
            numeric_imputation_target="median",
            # ignore_features = ['id', 'family', 'store_nbr']
          )  


In [ ]:
# check statistical tests on original data
check_stats()

sources for add_metric()

- https://towardsdatascience.com/predict-customer-churn-the-right-way-using-pycaret-8ba6541608ac
- https://github.com/pycaret/pycaret/issues/3491
- https://github.com/pycaret/pycaret/issues/1063

In [ ]:
from sklearn.metrics import mean_squared_log_error

# create a custom function
def rmsle(y_true, y_pred):
    return mean_squared_log_error(y_true
                        , y_pred
                        , squared=False)

add_metric('msle', 'MSLE', mean_squared_log_error, greater_is_better=False) # default squared=True
add_metric('rmsle', 'RMSLE', rmsle, greater_is_better=False) # for problem statement squared=False

In [ ]:
# compare baseline models
best = compare_models()